In [1]:
from pymongo import MongoClient
from bson.json_util import dumps
import pymongo
import pandas as pd
import requests
import nltk
mongodbURL = "mongodb://localhost/Chat_Sentiment_Analysis_Service"
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/david/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
def value_conversation(conversation):
    sia = SentimentIntensityAnalyzer()
    valorations=pd.DataFrame(columns=["neg","neu","pos","compound"])
    sentences=requests.get(f"http://127.0.0.1:4000/chat/{conversation}/list").json()
    for sentence in sentences:
        scores=sia.polarity_scores(list(sentence.values())[0])
        scores_df=pd.DataFrame(data=[scores.values()],columns=["neg","neu","pos","compound"])
        valorations=pd.concat([valorations,scores_df])
    return(valorations.mean().to_json())

In [3]:
def value_user(user):
    messages_id=db["users"].find_one({"username":user},{"messages":1})
    messages=[db.messages.find_one({"_id":ident},{"message":1})["message"] for ident in messages_id["messages"]]
    sia = SentimentIntensityAnalyzer()
    valorations=pd.DataFrame(columns=["neg","neu","pos","compound"])
    for sentence in messages:
        scores=sia.polarity_scores(sentence)
        scores_df=pd.DataFrame(data=[scores.values()],columns=["neg","neu","pos","compound"])
        valorations=pd.concat([valorations,scores_df])
    return(valorations.mean())

In [4]:
def create_df_with_puntuations():
    users=[user["username"] for user in list(db.users.find({},{"username":1}))]
    valorations=pd.DataFrame(columns=["neg","neu","pos","compound"])
    for user in users:
        valoration=pd.DataFrame(data=[value_user(user)],index=[user])
        valorations=pd.concat([valorations,valoration])
    valorations.to_csv("valorations.csv")
    return (valorations)

In [8]:
puntuations=pd.read_csv("../../output/valorations.csv")

,Unnamed: 0,neg,neu,pos,compound
0,kinghenryiv,0.082659,0.825774,0.091573,0.019065
1,westmoreland,0.089622,0.825341,0.085037,-0.017067
2,falstaff,0.079114,0.824924,0.095964,0.019717
3,princehenry,0.081523,0.815655,0.102832,0.028733
4,poins,0.077795,0.875977,0.046239,-0.012797
...,...,...,...,...,...
915,florizel,0.050370,0.833216,0.116423,0.079198
916,perdita,0.063821,0.814134,0.122052,0.074671
917,dorcas,0.048917,0.794917,0.156167,0.124775
918,mopsa,0.012600,0.785150,0.202250,0.214050


In [52]:
find_closest_users("titania")

{'closest_users': 'hamlet , curtis , thaliard'}